## 라이브러리 설치

In [ ]:
!pip install faiss-cpu ##또는 faiss-gpu
#tall -U accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.0 MB/s eta 0:00:00


# OpenAI Assistant 쓰기

In [ ]:
import os
from openai import OpenAI
import time
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
OPENAI_KEY = ""

client = OpenAI(api_key=OPENAI_KEY)

In [ ]:
# 문서가 저장된 폴더 경로
DOCUMENTS_PATH = "./doc_example"

# 폴더에서 .txt 파일 읽어오기
def load_documents_from_folder(folder_path):
    documents = []
    filenames = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # .txt 파일만 로드
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                content = file.read().strip()
                documents.append(content)
                filenames.append(filename)

    return documents, filenames

# 문서 로드
documents, filenames = load_documents_from_folder(DOCUMENTS_PATH)
# 문서 임베딩 모델 로드
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# 문서 임베딩 생성
document_embeddings = embedding_model.encode(documents)

# FAISS 인덱스 생성
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def retrieve_relevant_documents(query, top_k=1):
    """질문에 가장 관련 있는 문서를 검색"""
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [documents[idx] for idx in indices[0]]

In [ ]:
user_input = input("질문의 주제를 입력하세요: ")
relevant_docs = retrieve_relevant_documents(user_input)
print(relevant_docs)

In [ ]:
assistant = client.beta.assistants.create(
    name="ChatPDF",
    instructions="입력된 내용을 바탕으로 사용자의 질문에 답하라라",
    #tools=[{"type": "file_search"}],
    model="gpt-4-1106-preview" ## 모델 변경 가능.
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=relevant_docs[0]
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id = assistant.id,
)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
assistant_response = messages.data[0].content  # 최신 메시지
print(f"Assistant: {assistant_response}")

Assistant: []


In [ ]:
while True:
    user_input = input("You: ")
    if user_input == "0":  # 탈출 조건
        print("Chatbot 종료.")
        break

    # 사용자 메시지 추가
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # 메시지를 처리하는 run 실행
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id = assistant.id
    )

    # run이 완료될 때까지 대기
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        if run.status == "completed":
            break
        time.sleep(5)  # 1초 대기 후 다시 확인

    # 응답 메시지 가져오기
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    assistant_response = messages.data[0].content  # 최신 메시지 가져오기
    # 텍스트 값만 추출
    response_text = assistant_response[0].text.value
    print(f"Assistant: {response_text}")